# Demo de dos modelos de lengua

<a target="_blank" href="https://colab.research.google.com/github/jaspock/me/blob/main/docs/materials/assets/misterios/notebooks/llama.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a> <a href="http://dlsi.ua.es/~japerez/"><img src="https://img.shields.io/badge/Universitat-d'Alacant-5b7c99" style="margin-left:10px"></a>

Este cuaderno permite evaluar las continuaciones y las probabilidades de la siguiente palabra según dos modelos de lengua, uno entrenado únicamente para predecir el siguiente token y otro entrenado para seguir instrucciones.

El entorno de ejecución de este cuaderno ha de tener una GPU. La puedes conseguir desde el menú *Entorno de ejecución* / *Cambiar tipo de entorno de ejecución*.

## Inicialización de los modelos

El siguiente código inicializa el modelo base y el modelo que sigue instrucciones a partir de los modelos abiertos de tamaño 1B (1 millardo de parámetros) de la familia [Llama-3.2](https://huggingface.co/collections/meta-llama/llama-32-66f448ffc8c32f949b04c8cf) liberados por [Meta](https://ai.meta.com/blog/meta-llama-quantized-lightweight-models/). Para que funcione, has de tener una clave secreta de nombre `HF_TOKEN` y con un valor que puedes obtener si te creas una cuenta en [Hugging Face](https://huggingface.co/). Esta clave se añade desde la sección con el icono de la llave a la izquierda de este cuaderno.

La descarga de los modelos puede llevar unos minutos, especialmente la primera vez que se ejecute el cuaderno en un nuevo entorno de ejecución.

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Base model setup
# base_model_id = "meta-llama/Llama-3.2-3B"
base_model_id = "meta-llama/Llama-3.2-1B"
base_pipe = pipeline(
    "text-generation",
    model=base_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
base_tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                  torch_dtype=torch.bfloat16, device_map="auto")

# Instruction-tuned model setup
# instruction_model_id = "meta-llama/Llama-3.2-3B-Instruct"
instruction_model_id = "meta-llama/Llama-3.2-1B-Instruct"
instruction_pipe = pipeline(
    "text-generation",
    model=instruction_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
instruction_tokenizer = AutoTokenizer.from_pretrained(instruction_model_id)
instruction_model = AutoModelForCausalLM.from_pretrained(instruction_model_id,
                                                         torch_dtype=torch.bfloat16, device_map="auto")

## Obtención de las probabilidades de salida

La función `get_word_probs` devuelve un diccionario con las probabilidades del modelo para las palabras de la lista `candidates` cuando se ha procesado el contexto `text`.

In [ ]:
def get_word_probs(model, tokenizer, text, candidates):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model(**inputs)
    logits = outputs.logits[0, -1]  # last token logits
    probs = torch.softmax(logits, dim=-1)

    candidate_probs = {}  # initial dictionary
    for word in candidates:
        token_id = tokenizer.convert_tokens_to_ids(word)
        if token_id is not None:
            candidate_probs[word] = probs[token_id].item()
        else:
            candidate_probs[word] = 0.0  # word not in embedding table

    return candidate_probs

## Probando, probando...

En primer lugar, probaremos que los modelos funcionan pidiéndoles que continúen un texto.

Si el valor de `do_sample` está a `True`, el sistema generaría salidas diferentes cada vez. La diversidad en ese caso se puede ajustar jugando con los argumentos siguientes:

- `top_k`: limita el número de tokens más probables a considerar; por ejemplo, `top_k=50`
- `top_p`: elige tokens hasta alcanzar una probabilidad acumulada específica; por ejemplo, `top_p=0.95`
- `temperature`: controla la aleatoriedad: valores bajos entre 0 y 1 son más conservadores, altos entre 1 y 2 más creativos, pero aumentando el riesgo de incoherencias; por ejemplo, `temperature=0.7`.

In [ ]:
question = "What is the capital of France?"

inputs_normal = base_tokenizer(question, return_tensors="pt").to("cuda")
response_normal = base_model.generate(**inputs_normal, max_new_tokens=50)
print("🡪 Response of the base model:" , base_tokenizer.decode(response_normal[0], skip_special_tokens=True, do_sample=True))

inputs_instr = instruction_tokenizer(question, return_tensors="pt").to("cuda")
response_instr = instruction_model.generate(**inputs_instr, max_new_tokens=50)
print("🡪 Response of the instruction-tuned model: ", instruction_tokenizer.decode(response_instr[0], skip_special_tokens=True, do_sample=True))

## Obtención de probabilidades

A continuación, usaremos los modelos para obtener las probabilidades de la siguiente palabra tras el texto de `example-text` para aquellas palabras de la lista `candidate_words`.

En realidad, los modelos de lengua como Llama-3.2 no trabajan necesariamente a nivel de palabra, sino que por ciertos motivos que no vienen al caso, trocean las palabras pocos frecuentes en fragmentos que sí son frecuentes. A los elementos del vocabulario resultantes (que pueden ser palabras completas o fragmentos de ellas) se les denomina *tokens*.

Por ello, asegúrate de que `candidate_words` contiene palabras que sí están en el vocabulario; una manera rápida de comprobarlo es verificar que no se devuelve 0.0 para su probabilidad, ya que es lo que hace la función si la palabra no está en el vocabulario. Aunque en un principio podría parecer que esto no nos permite diferenciar entre palabras que no están en el vocabulario y palabras que sí lo están pero tienen probabilidad nula, lo cierto es que por las funciones matemáticas que usa la red neuronal, la salida de las palabras poco probables será un valor muy pequeño, pero nunca cero.

Observa que las probabilidades se presentan en notación científica, por ejemplo, `7.59027898311615e-08`. Esto equivale a $7.59027898311615 \times 10^{-8}$ o, lo que es lo mismo, 0.0000000759027898311615.

In [ ]:
example_text = "The weather in this summer day is "
candidate_words = ["hot", "cold", "nice", "car"]

probs_base = get_word_probs(base_model, base_tokenizer, example_text, candidate_words)
print("🡪 Probabilities of the base model:", probs_base)

instruction_text = "Complete the sentence: '" + example_text + "'"
probs_instr = get_word_probs(instruction_model, instruction_tokenizer, instruction_text, candidate_words)
print("🡪 Probabilities of the instruction-tuned model:", probs_instr)

## Generación de respuestas

Por último, generaremos más salidas. En el caso del modelo que sigue instrucciones, usaremos la plantilla que se usó durante su entrenamiento para obtener mejores resultados.

Para las personas muy observadoras, en el código anterior hemos usado por separado el *tokenizador* (que obtiene los *tokens* asociados a las palabras) y el modelo en sí. En el código siguiente, usaremos un *pipeline* que simplifica el código para obtener la salida del modelo, pero no nos permite obtener de forma directa las probabilidades de salida que queríamos antes.

In [ ]:
output = base_pipe("The key to life is", max_new_tokens=50, do_sample=True)
print("🡪 Response of the base model: ", output[0]["generated_text"])

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
outputs = instruction_pipe(messages, max_new_tokens=50, do_sample=True)
print("🡪 Response of the instruction-tuned model: ", outputs[0]["generated_text"][-1])   # print only new output

# call apply_chat_template if not using the pipeline:
# prompt = base_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)